In [13]:
dir_ = 'C:/Users/kimms/OneDrive/바탕 화면/소캡디/' # input only here

#### setting other directory

In [14]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [16]:
ver, KKK = 'priv', 0
STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [17]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMClassifier
import time


warnings.filterwarnings('ignore')

In [18]:
################################## SEED 설정 ####################################
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [19]:
############### Target Store에 해당하는 Data 로드############################

def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[df['store_id']==store]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3

    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


#################### 전처리를 위한 (이동평균선) 데이터 전처리 코드 ################################33

def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

### 하이퍼 파라미터 튜닝 후 refit 이용해 재학습

In [20]:
########################### Default model parameter설정 (기준값) ##########################

import lightgbm as lgb
lgb_params = {} 

In [21]:
########################### 전처리 변수 생성 및 데이터 불러오기 #########################

VER = 1                          
SEED = 42                        
seed_everything(SEED)            
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

#### 모든 Store에 대해서 하이퍼 파라미터 튜닝 

In [ ]:
T_STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [ ]:
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves',2**9,2**10,5),
                        'max_depth': hp.quniform('max_depth',80,140,1),
                     'min_data_in_leaf': hp.quniform('min_data_in_leaf',2**10,2**11,10),
                        'subsample':hp.uniform('subsample', 0.5,1),
                        'learning_rate': hp.uniform('learning_rate',0.01,0.2)}

def objective_func(search_space):
    lgbm_clf = LGBMRegressor(n_estimators = 1000,
                              boosting_type ='gbdt',
                              objective='tweedie',
                              tweedie_variance_power=1.1,
                              metric='rmse',
                              boost_from_average=False, 
                              feature_fraction=0.5,
                              max_bin=100,
                              num_leaves = int(search_space['num_leaves']),
                              max_depth= int(search_space['max_depth']),
                              min_data_in_leaf = int(search_space['min_data_in_leaf']),
                              subsample = search_space['subsample'],
                              learning_rate = search_space['learning_rate'],
                              n_jobs = -1)
    
    
    
    lgbm_clf.fit(x_train,y_train)
    pred = lgbm_clf.predict(latest_x_valid)
    rms = sqrt(mean_squared_error(latest_y_valid, pred))
    print("valid: ",rms)
    
    return rms

In [ ]:
for Teacher_Store in tqdm(T_STORES_IDS):
    ########################### Train Teacher Models
    #################################################################################
    print('Teacher Train', Teacher_Store)
    store_id = Teacher_Store
    best = parameter_dict[Teacher_Store]
    lgb_params = {  
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': round(best['subsample'],5),
                    'subsample_freq': 1,
                    'learning_rate': round(best['learning_rate'],5),
                    'num_leaves': int(best['num_leaves']),
                    'min_data_in_leaf': int(best['min_data_in_leaf']),
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'max_depth':int(best['max_depth'])
                } 

    grid_df, features_columns = get_data_by_store(store_id)

    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

    # 최근 약 3년의 데이터만 사용 
    latest_1y = int(len(train_mask)/5)*3

    train_data = lgb.Dataset(grid_df[train_mask][features_columns].iloc[-latest_1y:], 
                       label=grid_df[train_mask][TARGET].iloc[-latest_1y:])

    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])


    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]

    d_sales = grid_df[['d','sales']]
    substitute = d_sales['sales'].values
    substitute[(d_sales['d'] > END_TRAIN)] = np.nan
    grid_df['sales'] = substitute

    grid_df.to_pickle(processed_data_dir+'test_'+store_id+'.pkl')
    del grid_df, d_sales, substitute

    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )


    display(pd.DataFrame({'name':estimator.feature_name(),
                          'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))


    model_name = model_dir+'lgb_model_3y_T_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    del train_data, valid_data, estimator
    gc.collect()

    MODEL_FEATURES = features_columns

In [ ]:
for Teacher_Store in tqdm(STORES_IDS):
    ########################### Train Teacher Models
    #################################################################################
    print('Teacher Train', Teacher_Store)
    store_id = Teacher_Store
    best = oney_parameter_dict[Teacher_Store]
    lgb_params = {  
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': round(best['subsample'],5),
                    'subsample_freq': 1,
                    'learning_rate': round(best['learning_rate'],5),
                    'num_leaves': int(best['num_leaves']),
                    'min_data_in_leaf': int(best['min_data_in_leaf']),
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'max_depth':int(best['max_depth'])
                } 

    grid_df, features_columns = get_data_by_store(store_id)

    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

    # 최근 약 3년의 데이터만 사용 
    latest_1y = int(len(train_mask)/5)*3

    train_data = lgb.Dataset(grid_df[train_mask][features_columns].iloc[-latest_1y:], 
                       label=grid_df[train_mask][TARGET].iloc[-latest_1y:])

    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])


    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]

    d_sales = grid_df[['d','sales']]
    substitute = d_sales['sales'].values
    substitute[(d_sales['d'] > END_TRAIN)] = np.nan
    grid_df['sales'] = substitute

    grid_df.to_pickle(processed_data_dir+'test_'+store_id+'.pkl')
    del grid_df, d_sales, substitute

    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )


    display(pd.DataFrame({'name':estimator.feature_name(),
                          'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))


    model_name = model_dir+'lgb_model_1y_hy_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    del train_data, valid_data, estimator
    gc.collect()

    MODEL_FEATURES = features_columns

## CA

    CA 1부터 4까지 각각 Teacher model로 잡고 각 경우에서 나머지 9개 store에 대해서 refit을 사용해서 재학습 진행 및 저장 

In [10]:
T_STORES_IDS = ['CA_1','CA_2','CA_3','CA_4']
S_STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [ ]:
for t_store_id in tqdm(T_STORES_IDS):
    Teacher_Store = t_store_id
    print("Teacher store: ", Teacher_Store)
    
    for store_id in tqdm(S_STORES_IDS):
        
        # 학습된 Teacher model
        model_path = model_dir+'lgb_model_3y_T_'+Teacher_Store+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path

        T_model = pickle.load(open(model_path, 'rb'))
        
        if t_store_id == store_id:
            # teacher model에 대해서는 재학습이 아닌 그대로 모델을 사용 
            model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
            pickle.dump(T_model, open(model_name, 'wb'))
            continue
            
        print('Train', store_id)
        grid_df, features_columns = get_data_by_store(store_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        # 학습 데이터 정의 
        x_train = grid_df[train_mask][features_columns]
        y_train = grid_df[train_mask][TARGET]

        # 최근 약 1년의 데이터만 사용 
        latest_1y = int(len(train_mask)/5)

        x_train = x_train.iloc[-latest_1y:]
        y_train = y_train.iloc[-latest_1y:]


        # valid 데이터 정의 
        full_x_valid = grid_df[valid_mask][features_columns]
        full_y_valid = grid_df[valid_mask][TARGET]

        latest_x_valid = full_x_valid.iloc[-latest_1y:]
        latest_y_valid = full_y_valid.iloc[-latest_1y:]

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        del grid_df, d_sales, substitute


        seed_everything(SEED)
        S_model = T_model.refit(data=x_train, label=y_train, decay_rate=0)
        y_pred = S_model.predict(full_x_valid)
        score = mean_squared_error(full_y_valid, y_pred)**0.5
        print("Full period RMSE: ",score)

        y_pred = S_model.predict(latest_x_valid)
        score = mean_squared_error(latest_y_valid, y_pred)**0.5
        print("latest period RMSE: ",score)

        display(pd.DataFrame({'name':S_model.feature_name(),
                              'imp':S_model.feature_importance()}).sort_values('imp',ascending=False).head(25))


        model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
        pickle.dump(S_model, open(model_name, 'wb'))


        del train_data, valid_data, T_model
        gc.collect()

        MODEL_FEATURES = features_columns


## TX

    TX 1부터 3까지 각각 Teacher model로 잡고 각 경우에서 나머지 9개 store에 대해서 refit을 사용해서 재학습 진행 및 저장 

In [ ]:
T_STORES_IDS = ['TX_1','TX_2','TX_3']
S_STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [ ]:
for t_store_id in tqdm(T_STORES_IDS):
    Teacher_Store = t_store_id
    print("Teacher store: ", Teacher_Store)
    
    for store_id in tqdm(S_STORES_IDS):
        
        # 학습된 Teacher model
        model_path = model_dir+'lgb_model_3y_T_'+Teacher_Store+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path

        T_model = pickle.load(open(model_path, 'rb'))
        
        if t_store_id == store_id:
            # teacher model에 대해서는 재학습이 아닌 그대로 모델을 사용 
            model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
            pickle.dump(T_model, open(model_name, 'wb'))
            continue
            
        print('Train', store_id)
        grid_df, features_columns = get_data_by_store(store_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        # 학습 데이터 정의 
        x_train = grid_df[train_mask][features_columns]
        y_train = grid_df[train_mask][TARGET]

        # 최근 약 1년의 데이터만 사용 
        latest_1y = int(len(train_mask)/5)

        x_train = x_train.iloc[-latest_1y:]
        y_train = y_train.iloc[-latest_1y:]


        # valid 데이터 정의 
        full_x_valid = grid_df[valid_mask][features_columns]
        full_y_valid = grid_df[valid_mask][TARGET]

        latest_x_valid = full_x_valid.iloc[-latest_1y:]
        latest_y_valid = full_y_valid.iloc[-latest_1y:]

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        del grid_df, d_sales, substitute


        seed_everything(SEED)
        S_model = T_model.refit(data=x_train, label=y_train, decay_rate=0)
        y_pred = S_model.predict(full_x_valid)
        score = mean_squared_error(full_y_valid, y_pred)**0.5
        print("Full period RMSE: ",score)

        y_pred = S_model.predict(latest_x_valid)
        score = mean_squared_error(latest_y_valid, y_pred)**0.5
        print("latest period RMSE: ",score)

        display(pd.DataFrame({'name':S_model.feature_name(),
                              'imp':S_model.feature_importance()}).sort_values('imp',ascending=False).head(25))


        model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
        pickle.dump(S_model, open(model_name, 'wb'))


        del train_data, valid_data, T_model
        gc.collect()

        MODEL_FEATURES = features_columns


## WI

    WI 1부터 3까지 각각 Teacher model로 잡고 각 경우에서 나머지 9개 store에 대해서 refit을 사용해서 재학습 진행 및 저장 

In [ ]:
T_STORES_IDS = ['WI_1','WI_2','WI_3']
S_STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

In [ ]:
for t_store_id in tqdm(T_STORES_IDS):
    Teacher_Store = t_store_id
    print("Teacher store: ", Teacher_Store)
    
    for store_id in tqdm(S_STORES_IDS):
        
        # 학습된 Teacher model
        model_path = model_dir+'lgb_model_3y_T_'+Teacher_Store+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path

        T_model = pickle.load(open(model_path, 'rb'))
        
        if t_store_id == store_id:
            # teacher model에 대해서는 재학습이 아닌 그대로 모델을 사용 
            model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
            pickle.dump(T_model, open(model_name, 'wb'))
            continue
            
        print('Train', store_id)
        grid_df, features_columns = get_data_by_store(store_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        # 학습 데이터 정의 
        x_train = grid_df[train_mask][features_columns]
        y_train = grid_df[train_mask][TARGET]

        # 최근 약 1년의 데이터만 사용 
        latest_1y = int(len(train_mask)/5)

        x_train = x_train.iloc[-latest_1y:]
        y_train = y_train.iloc[-latest_1y:]


        # valid 데이터 정의 
        full_x_valid = grid_df[valid_mask][features_columns]
        full_y_valid = grid_df[valid_mask][TARGET]

        latest_x_valid = full_x_valid.iloc[-latest_1y:]
        latest_y_valid = full_y_valid.iloc[-latest_1y:]

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        del grid_df, d_sales, substitute


        seed_everything(SEED)
        S_model = T_model.refit(data=x_train, label=y_train, decay_rate=0)
        y_pred = S_model.predict(full_x_valid)
        score = mean_squared_error(full_y_valid, y_pred)**0.5
        print("Full period RMSE: ",score)

        y_pred = S_model.predict(latest_x_valid)
        score = mean_squared_error(latest_y_valid, y_pred)**0.5
        print("latest period RMSE: ",score)

        display(pd.DataFrame({'name':S_model.feature_name(),
                              'imp':S_model.feature_importance()}).sort_values('imp',ascending=False).head(25))


        model_name = model_dir+'lgb_model_3y_'+Teacher_Store+'_'+store_id+'_v'+str(VER)+'.bin'
        pickle.dump(S_model, open(model_name, 'wb'))


        del train_data, valid_data, T_model
        gc.collect()

        MODEL_FEATURES = features_columns
